# FASTVIT PET MOBILE

## 1. Environment setup

In [5]:
!git clone https://github.com/HenryNVP/fastvit-pet-mobile.git
%cd fastvit-pet-mobile

Cloning into 'fastvit-pet-mobile'...
remote: Enumerating objects: 13, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 13 (delta 0), reused 13 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (13/13), 10.04 KiB | 10.04 MiB/s, done.
/content/fastvit-pet-mobile/fastvit-pet-mobile


In [10]:
!pip install -r requirements.txt

ERROR: Could not find a version that satisfies the requirement torch==1.11.0 (from versions: 2.2.0, 2.2.1, 2.2.2, 2.3.0, 2.3.1, 2.4.0, 2.4.1, 2.5.0, 2.5.1, 2.6.0, 2.7.0, 2.7.1, 2.8.0, 2.9.0, 2.9.1)
ERROR: No matching distribution found for torch==1.11.0


## 2. Download and prepare data

This downloads the official dataset and prepares 256x256 splits.

In [11]:
!python scripts/get_data.py
!python scripts/split_dataset.py

/content/fastvit-pet-mobile/fastvit-pet-mobile/scripts/get_data.py:33: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
  tf.extractall(destination)
Prepared train=3680 val=1832 test=1837


## 3. Finetune teacher model

## 4. Setup and finetune student model

## 5. Model evaluation

## 6. Export model

## 7. Summary and conclusion